<a href="https://colab.research.google.com/github/BryanYuen0205/COMP4211_Kaggle_Comp/blob/main/COMP4211KaggleComp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Setting up environment
!python --version
!pip show torch tensorflow scikit-learn
!pip install --upgrade "tensorflow>=2.18"
!pip install "scikit-learn>=1.4.1"
!pip install kaggle


Python 3.11.12
Name: torch
Version: 2.6.0+cu124
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-cusparselt-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, fastai, peft, sentence-transformers, timm, torchaudio, torchvision
---
Name: tensorflow
Version: 2.19.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [14]:
# To safely store your training progress, use Google Drive:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
# To upload kaggle.json file
from google.colab import files
files.upload()


{}

In [30]:
# Creating a kaggle directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [32]:
# Downloading the kaggle competition dataset
!kaggle competitions download -c comp-4211-spring-25-project

In [33]:
!unzip -q comp-4211-spring-25-project.zip -d ./data

In [34]:
import pandas as pd

# Load the datasets
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

# View first few rows
print("Train Data:")
display(train_df.head())

print("Test Data:")
display(test_df.head())


Train Data:


,id,Sentence,NER Tag
0,0,"['Thousands', 'of', 'demonstrators', 'have', '...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,1,"['Iranian', 'officials', 'say', 'they', 'expec...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '..."
2,2,"['Helicopter', 'gunships', 'Saturday', 'pounde...","['O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', '..."
3,3,"['They', 'left', 'after', 'a', 'tense', 'hour-...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,4,"['U.N.', 'relief', 'coordinator', 'Jan', 'Egel...","['B-geo', 'O', 'O', 'B-per', 'I-per', 'O', 'B-..."


Test Data:


,id,Sentence
0,0,"['Pakistani', 'officials', 'say', 'unidentifie..."
1,1,"['China', ""'s"", 'official', 'Xinhua', 'news', ..."
2,2,"['News', 'reports', 'say', 'since', '1998', ',..."
3,3,"['French', 'tennis', 'great', 'Mary', 'Pierce'..."
4,4,"['Now', 'ranked', '29th', 'in', 'the', 'world'..."
